In [ ]:
import pennylane as qml
from pennylane import numpy as numpy

qml.about()


ModuleNotFoundError: No module named 'pennylane'

In [ ]:
!pip install cirq
!pip install mitiq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.5/597.5 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.1/425.1 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninst

In [ ]:
import cirq
import numpy as np
from mitiq import zne, pec, cdr
from mitiq.zne.inference import LinearFactory, RichardsonFactory
from mitiq.zne.scaling import fold_gates_at_random
from mitiq.benchmarks import generate_rb_circuits
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq


# Generate a random benchmark circuit (following documentation)
circuit = generate_rb_circuits(n_qubits=1, num_cliffords=2, return_type="cirq")[0]
print(f"Original circuit:\n{circuit}")

# Define executor function (following documentation)
def execute(circuit, noise_level=0.01):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    executed with depolarizing noise.
    """
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[0, 0].real

# Compute baseline values
noisy_value = execute(circuit)
ideal_value = execute(circuit, noise_level=0.0)
print(f"Ideal value: {ideal_value:.5f}")
print(f"Noisy value: {noisy_value:.5f}")
print(f"Error without mitigation: {abs(ideal_value - noisy_value):.5f}")

print("\n--- Method 1: One-step ZNE ---")
# One-step ZNE (following documentation)
mitigated_result = zne.execute_with_zne(circuit, execute)
print(f"Mitigated value: {mitigated_result:.5f}")
print(f"Error with mitigation (ZNE): {abs(ideal_value - mitigated_result):.5f}")

print("\n--- Method 2: ZNE with custom factory ---")
# ZNE with custom factory (following documentation)
linear_fac = LinearFactory(scale_factors=[1.0, 2.0])
mitigated_result_custom = zne.execute_with_zne(
    circuit, execute, factory=linear_fac, scale_noise=fold_gates_at_random
)
print(f"Mitigated value (custom): {mitigated_result_custom:.5f}")
print(f"Error with custom ZNE: {abs(ideal_value - mitigated_result_custom):.5f}")

print("\n--- Method 3: Two-stage ZNE ---")
# Two-stage ZNE (following documentation exactly)
scale_factors = [1.0, 2.0, 3.0]

# Step 1: Construct circuits with scale_method (NOT factory)
folded_circuits = zne.construct_circuits(
    circuit=circuit,
    scale_factors=scale_factors,
    scale_method=fold_gates_at_random  # This is the correct parameter name
)

print(f"Number of folded circuits: {len(folded_circuits)}")
for i, circ in enumerate(folded_circuits):
    print(f"Circuit {i+1} (scale factor {scale_factors[i]}):")
    print(f"  Number of gates: {len(list(circ.all_operations()))}")

# Step 2: Execute all circuits
results = [execute(circuit) for circuit in folded_circuits]
print(f"Execution results: {results}")

# Step 3: Combine results using extrapolation
extrapolation_method = RichardsonFactory(scale_factors=scale_factors).extrapolate
two_stage_zne_result = zne.combine_results(
    scale_factors, results, extrapolation_method
)

print(f"Two-stage ZNE result: {two_stage_zne_result:.5f}")
print(f"Error with two-stage ZNE: {abs(ideal_value - two_stage_zne_result):.5f}")

print("\n=== Probabilistic Error Cancellation (PEC) ===")

# Define a simple circuit for PEC
pec_circuit = cirq.Circuit(
    cirq.H(cirq.LineQubit(0)),
    cirq.CNOT(cirq.LineQubit(0), cirq.LineQubit(1))
)

try:
    # Note: PEC requires proper noise representations which are complex to set up
    print("PEC requires detailed noise model setup - skipping for this example")
    print("Refer to Mitiq documentation for complete PEC implementation")
except Exception as e:
    print(f"PEC construction failed: {e}")

print("\n=== Clifford Data Regression (CDR) ===")

# Generate training circuits for CDR
training_circuits = generate_rb_circuits(n_qubits=2, num_cliffords=20, trials=5, return_type="cirq")
target_circuit = generate_rb_circuits(n_qubits=2, num_cliffords=30, return_type="cirq")[0]

try:
    # Construct CDR circuits
    cdr_circuits = cdr.construct_circuits(
        target_circuit,
        training_circuits=training_circuits
    )

    print(f"Number of CDR circuits: {len(cdr_circuits)}")
    print(f"Training circuits: {len(training_circuits)}")
    print(f"Target circuit depth: {len(target_circuit)}")

except Exception as e:
    print(f"CDR construction failed: {e}")

print("\n=== Summary ===")
print("ZNE successfully demonstrated with three methods:")
print("1. One-step execute_with_zne()")
print("2. Custom factory with execute_with_zne()")
print("3. Two-stage construct_circuits() -> combine_results()")
print("\nKey correction: construct_circuits() uses 'scale_method', not 'factory'")

=== Zero Noise Extrapolation (ZNE) ===
Original circuit:
0: ───Y^-0.5───X^-0.5───Y───X^0.5───H───
Ideal value: 1.00000
Noisy value: 0.96754
Error without mitigation: 0.03246

--- Method 1: One-step ZNE ---
Mitigated value: 0.98214
Error with mitigation (ZNE): 0.01786

--- Method 2: ZNE with custom factory ---
Mitigated value (custom): 0.99199
Error with custom ZNE: 0.00801

--- Method 3: Two-stage ZNE ---
Number of folded circuits: 3
Circuit 1 (scale factor 1.0):
  Number of gates: 5
Circuit 2 (scale factor 2.0):
  Number of gates: 9
Circuit 3 (scale factor 3.0):
  Number of gates: 15
Execution results: [0.9675438, 0.9431022, 0.9088148]
Two-stage ZNE result: 0.98214
Error with two-stage ZNE: 0.01786

=== Probabilistic Error Cancellation (PEC) ===
PEC requires detailed noise model setup - skipping for this example
Refer to Mitiq documentation for complete PEC implementation

=== Clifford Data Regression (CDR) ===
CDR construction failed: module 'mitiq.cdr' has no attribute 'construct_ci

In [ ]:
def analyze_gate_overhead(circuit):
    operations = list(circuit.all_operations())  # Convert generator to list
    total_gates = len(operations)
    two_qubit_gates = len([op for op in operations if len(op.qubits) == 2])
    single_qubit_gates = total_gates - two_qubit_gates

    print(f"Total gates: {total_gates}")
    print(f"2-qubit gates: {two_qubit_gates}")
    print(f"1-qubit gates: {single_qubit_gates}")
    if total_gates > 0:
        print(f"2-qubit gate ratio: {two_qubit_gates/total_gates:.2%}")
    else:
        print("No gates in the circuit.")

# example usage
# Note: generate_rb_circuits returns a list of circuits, so we take the first one
circuit = generate_rb_circuits(n_qubits=2, num_cliffords=50)[0]
analyze_gate_overhead(circuit)

Total gates: 499
2-qubit gates: 75
1-qubit gates: 424
2-qubit gate ratio: 15.03%


In [ ]:
import cirq
import numpy as np
from mitiq import zne
from mitiq.zne.scaling import fold_gates_at_random
from mitiq.zne.inference import LinearFactory
from mitiq.benchmarks import generate_rb_circuits
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq

print(" Stage 1: Circuit Construction")
circuit = generate_rb_circuits(n_qubits=2, num_cliffords=50, return_type="cirq")[0]
print(f"Original circuit depth: {len(circuit)}")

scale_factors = [1.0, 2.0, 3.0]

zne_circuits = zne.construct_circuits(
    circuit,
    scale_factors=scale_factors,
    scale_method=fold_gates_at_random
)

print(f"Number of ZNE circuits generated: {len(zne_circuits)}")
for i, circ in enumerate(zne_circuits):
    print(f"Circuit {i+1} (scale factor {scale_factors[i]}): {len(circ)} operations")

print("\n Stage 2: Circuit Execution ")

def execute_circuit(circuit, noise_level=0.01):
    # Convert to mitiq format and add noise
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))

    # Simulate and get density matrix
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix

    # Return expectation value of |0...0><0...0| (ground state probability)
    ground_state_prob = rho[0, 0].real
    return ground_state_prob

# Execute all scaled circuits
results = [execute_circuit(circ) for circ in zne_circuits]
print(f"Execution results: {[f'{r:.4f}' for r in results]}")

# Stage 3: Result Combination

linear_factory = LinearFactory(scale_factors=scale_factors)
mitigated_result = zne.combine_results(
    scale_factors,           # scale_factors first
    results,                 # results second
    linear_factory.extrapolate  # extrapolation method
)

print(f"Mitigated result: {mitigated_result:.6f}")

# Compare with unmitigated result
unmitigated_result = execute_circuit(circuit)
ideal_result = execute_circuit(circuit, noise_level=0.0)

print(f"\n Comparison between the mitigated and the unmitigated result  ")
print(f"Ideal result (no noise): {ideal_result:.6f}")
print(f"Unmitigated result: {unmitigated_result:.6f}")
print(f"Mitigated result (ZNE): {mitigated_result:.6f}")
print(f"Error without mitigation: {abs(ideal_result - unmitigated_result):.6f}")
print(f"Error with ZNE mitigation: {abs(ideal_result - mitigated_result):.6f}")

# Alternative: One-step ZNE for comparison
one_step_result = zne.execute_with_zne(
    circuit,
    execute_circuit,
    factory=LinearFactory(scale_factors=[1.0, 2.0, 3.0]),
    scale_noise=fold_gates_at_random
)
print(f"One-step ZNE result: {one_step_result:.6f}")
print(f"Error with one-step ZNE: {abs(ideal_result - one_step_result):.6f}")

 Stage 1: Circuit Construction
Original circuit depth: 317
Number of ZNE circuits generated: 3
Circuit 1 (scale factor 1.0): 317 operations
Circuit 2 (scale factor 2.0): 693 operations
Circuit 3 (scale factor 3.0): 951 operations

 Stage 2: Circuit Execution 
Execution results: ['0.2508', '0.2500', '0.2500']
Mitigated result: 0.251085

 Comparison between the mitigated and the unmitigated result  
Ideal result (no noise): 1.000000
Unmitigated result: 0.250813
Mitigated result (ZNE): 0.251085
Error without mitigation: 0.749187
Error with ZNE mitigation: 0.748915
One-step ZNE result: 0.251085
Error with one-step ZNE: 0.748915


In [ ]:
import time
import cirq
import numpy as np
from mitiq import zne, pec, cdr
from mitiq.zne.scaling import fold_gates_at_random
from mitiq.zne.inference import LinearFactory
from mitiq.benchmarks import generate_rb_circuits
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq

def execute_circuit(circuit, noise_level=0.01):
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[0, 0].real

def benchmark_zne(circuit, scale_factors=[1.0, 2.0, 3.0]):
    start_time = time.time()

    circuits = zne.construct_circuits(circuit, scale_factors=scale_factors, scale_method=fold_gates_at_random)
    results = [execute_circuit(circ) for circ in circuits]
    factory = LinearFactory(scale_factors=scale_factors)
    mitigated_result = zne.combine_results(scale_factors, results, factory.extrapolate)

    return {
        'technique': 'ZNE',
        'time': time.time() - start_time,
        'circuits': len(circuits),
        'result': mitigated_result
    }

def benchmark_pec(circuit, num_samples=20):
    start_time = time.time()
    try:
        from mitiq.pec.representations.depolarizing import represent_operation_with_local_depolarizing_noise
        gate_types = {type(op.gate) for op in circuit.all_operations()}
        representations = {gt: represent_operation_with_local_depolarizing_noise for gt in gate_types}

        circuits = pec.construct_circuits(circuit, representations=representations, num_samples=num_samples)
        results = [execute_circuit(circ) for circ in circuits]
        mitigated_result = np.mean(results)

        return {'technique': 'PEC', 'time': time.time() - start_time, 'circuits': len(circuits), 'result': mitigated_result}
    except Exception as e:
        return {'technique': 'PEC', 'time': time.time() - start_time, 'circuits': 0, 'result': None, 'error': str(e)}

def benchmark_cdr(circuit, training_circuits):
    start_time = time.time()
    try:
        circuits = cdr.construct_circuits(circuit, training_circuits=training_circuits)
        results = [execute_circuit(circ) for circ in circuits]
        mitigated_result = results[0] if results else 0.5

        return {'technique': 'CDR', 'time': time.time() - start_time, 'circuits': len(circuits), 'result': mitigated_result}
    except Exception as e:
        return {'technique': 'CDR', 'time': time.time() - start_time, 'circuits': 0, 'result': None, 'error': str(e)}

# Setup
circuit = generate_rb_circuits(n_qubits=2, num_cliffords=50, return_type="cirq")[0]
training_circuits = generate_rb_circuits(n_qubits=2, num_cliffords=20, trials=3, return_type="cirq")

# Baseline
ideal_result = execute_circuit(circuit, noise_level=0.0)
noisy_result = execute_circuit(circuit, noise_level=0.01)

print("QEM Benchmark Results ")
print(f"Ideal: {ideal_result:.4f}, Noisy: {noisy_result:.4f}, Error: {abs(ideal_result - noisy_result):.4f}")

# Run benchmarks
benchmarks = [
    benchmark_zne(circuit),
    benchmark_pec(circuit),
    benchmark_cdr(circuit, training_circuits)
]

# Results
print(f"\n{'Method':<6} {'Time(s)':<8} {'Circuits':<9} {'Result':<8} {'Error':<8}")
print("-" * 50)
for b in benchmarks:
    if b['result'] is not None:
        error = abs(ideal_result - b['result'])
        print(f"{b['technique']:<6} {b['time']:.3f}    {b['circuits']:<9} {b['result']:.4f}   {error:.4f}")
    else:
        print(f"{b['technique']:<6} {b['time']:.3f}    {b['circuits']:<9} Failed   Failed")

print(f"\nBaseline error: {abs(ideal_result - noisy_result):.4f}")
print("ZNE: 3x overhead, reliable. PEC: High overhead, complex. CDR: Needs training data.")

=== QEM Benchmark Results ===
Ideal: 1.0000, Noisy: 0.2506, Error: 0.7494

Method Time(s)  Circuits  Result   Error   
--------------------------------------------------
ZNE    6.574    3         0.2507   0.7493
PEC    0.073    0         Failed   Failed
CDR    0.000    0         Failed   Failed

Baseline error: 0.7494
ZNE: 3x overhead, reliable. PEC: High overhead, complex. CDR: Needs training data.


In [ ]:
import cirq
import time
from mitiq import zne
from mitiq.zne.scaling import fold_gates_at_random
from mitiq.zne.inference import LinearFactory
from mitiq.benchmarks import generate_rb_circuits
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq

def robust_execute(circuit, noise_level=0.01, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Convert and add noise
            mitiq_circuit, _ = convert_to_mitiq(circuit)
            noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))

            # Execute with density matrix simulator
            rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
            result = rho[0, 0].real

            return result

        except Exception as e:
            if attempt == max_retries - 1:
                print(f"All {max_retries} attempts failed. Last error: {str(e)}")
                raise e
            print(f"Attempt {attempt + 1} failed: {str(e)}. Retrying...")
            time.sleep(0.1)  # Brief pause before retry

def execute_with_zne_robust(circuit, scale_factors=[1.0, 2.0, 3.0], max_retries=3):
    try:
        print("Constructing ZNE circuits...")

        # CORRECTED: construct_circuits needs scale_method, not factory
        zne_circuits = zne.construct_circuits(
            circuit,
            scale_factors=scale_factors,
            scale_method=fold_gates_at_random
        )

        print(f"Generated {len(zne_circuits)} circuits with scale factors {scale_factors}")

        # Execute each circuit with retry logic
        results = []
        for i, circ in enumerate(zne_circuits):
            print(f"Executing circuit {i+1}/{len(zne_circuits)} (scale factor {scale_factors[i]})...")
            result = robust_execute(circ, max_retries=max_retries)
            results.append(result)
            print(f"  Result: {result:.6f}")

        # CORRECTED: combine_results signature
        factory = LinearFactory(scale_factors=scale_factors)
        mitigated_result = zne.combine_results(
            scale_factors,  # scale_factors first
            results,        # results second
            factory.extrapolate  # extrapolation method
        )

        return mitigated_result, results

    except Exception as e:
        print(f"Error during ZNE execution: {str(e)}")
        return None, []

# Example usage with comprehensive error handling
def main():
    try:

        # Generate test circuit
        circuit = generate_rb_circuits(n_qubits=2, num_cliffords=30, return_type="cirq")[0]
        print(f"Generated circuit with {len(circuit)} operations")

        # Get baseline results
        ideal_result = robust_execute(circuit, noise_level=0.0)
        noisy_result = robust_execute(circuit, noise_level=0.01)

        print(f"Ideal result: {ideal_result:.6f}")
        print(f"Noisy result: {noisy_result:.6f}")
        print(f"Error without mitigation: {abs(ideal_result - noisy_result):.6f}")

        # Execute ZNE with error handling
        mitigated_result, individual_results = execute_with_zne_robust(
            circuit,
            scale_factors=[1.0, 2.0, 3.0],
            max_retries=3
        )

        if mitigated_result is not None:
            print(f"\nZNE Results:")
            print(f"Individual results: {[f'{r:.6f}' for r in individual_results]}")
            print(f"Mitigated result: {mitigated_result:.6f}")
            print(f"Error with ZNE: {abs(ideal_result - mitigated_result):.6f}")

            # Performance comparison
            error_reduction = abs(ideal_result - noisy_result) - abs(ideal_result - mitigated_result)
            improvement = (error_reduction / abs(ideal_result - noisy_result)) * 100
            print(f"Error reduction: {error_reduction:.6f} ({improvement:.1f}% improvement)")
        else:
            print("ZNE execution failed completely.")

        # Alternative: One-step ZNE for comparison
        try:
            one_step_result = zne.execute_with_zne(
                circuit,
                lambda c: robust_execute(c, max_retries=1),  # Wrapper for compatibility
                factory=LinearFactory(scale_factors=[1.0, 2.0, 3.0]),
                scale_noise=fold_gates_at_random
            )
            print(f"One-step ZNE result: {one_step_result:.6f}")
            print(f"Error with one-step ZNE: {abs(ideal_result - one_step_result):.6f}")
        except Exception as e:
            print(f"One-step ZNE failed: {str(e)}")

    except Exception as e:
        print(f"Critical error in main execution: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Generated circuit with 183 operations
Ideal result: 1.000000
Noisy result: 0.266766
Error without mitigation: 0.733234
Constructing ZNE circuits...
Generated 3 circuits with scale factors [1.0, 2.0, 3.0]
Executing circuit 1/3 (scale factor 1.0)...
  Result: 0.266766
Executing circuit 2/3 (scale factor 2.0)...
  Result: 0.250252
Executing circuit 3/3 (scale factor 3.0)...
  Result: 0.250007

ZNE Results:
Individual results: ['0.266766', '0.250252', '0.250007']
Mitigated result: 0.272434
Error with ZNE: 0.727566
Error reduction: 0.005668 (0.8% improvement)
One-step ZNE result: 0.272434
Error with one-step ZNE: 0.727566


In [ ]:
pip uninstall -y qiskit qiskit-terra qiskit-aer qiskit-ibmq-provider qiskit-ibm-provider


Found existing installation: qiskit 2.0.2
Uninstalling qiskit-2.0.2:
  Successfully uninstalled qiskit-2.0.2
Found existing installation: qiskit-ibm-provider 0.11.0
Uninstalling qiskit-ibm-provider-0.11.0:
  Successfully uninstalled qiskit-ibm-provider-0.11.0


In [ ]:
!pip install qiskit qiskit-ibm-provider


In [ ]:
# !pip install qiskit qiskit-ibm-runtime --quiet
# uncomment  this for installing qiskit framework's runtime


from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumCircuit

service = QiskitRuntimeService()

# Step 3: Choose a backend (simulator or real hardware)
backend = service.backend("ibmq_qasm_simulator")  # Or 'ibm_kyoto', etc.

# Step 4: Define a circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# Step 5: Analyze cost
def analyze_ibm_costs(circuit):
    num_qubits = circuit.num_qubits
    depth = circuit.depth()

    # Dummy pricing model (IBM does not disclose real prices)
    base_cost = 0.0001
    qubit_cost = 0.00001
    depth_cost = 0.000001

    estimated_cost = base_cost + (num_qubits * qubit_cost) + (depth * depth_cost)

    return {
        "backend": backend.name,
        "num_qubits": num_qubits,
        "depth": depth,
        "estimated_cost_usd": round(estimated_cost, 8)
    }

# Step 6: Print results
print(analyze_ibm_costs(qc))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.9/363.9 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 11.8 MB/s eta 0:00:00


AccountNotFoundError: 'Unable to find account.'

In [ ]:
!pip install mitiq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 19.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you 

In [ ]:
!pip install cirq-rigetti mitiq --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
from mitiq.interface.mitiq_pyquil import pyquil_executor
from mitiq import convert_from_mitiq, convert_to_mitiq
from pyquil import Program
from pyquil.gates import *
from qiskit import QuantumCircuit
import warnings

def analyze_rigetti_costs(circuit):
    """
    Analyze costs for Rigetti execution.

    Args:
        circuit: Either a Mitiq circuit or Qiskit QuantumCircuit

    Returns:
        dict: Circuit characteristics and cost analysis
    """
    try:
        # Handle different input types
        if hasattr(circuit, 'num_qubits'):  # Qiskit circuit
            # Convert Qiskit to Mitiq first, then to PyQuil
            mitiq_circuit = convert_to_mitiq(circuit)
            pyquil_circuit = convert_from_mitiq(mitiq_circuit, "pyquil")
        else:
            # Assume it's already a Mitiq circuit
            pyquil_circuit = convert_from_mitiq(circuit, "pyquil")

        # Get circuit characteristics
        try:
            # Get qubits used in the circuit
            qubits_used = pyquil_circuit.get_qubits()
            num_qubits = len(qubits_used) if qubits_used else 0

            # Estimate circuit depth (number of instructions)
            circuit_depth = len(pyquil_circuit.instructions)

            # Count different types of gates
            gate_count = len([instr for instr in pyquil_circuit.instructions
                            if hasattr(instr, 'name')])

        except Exception as e:
            print(f"Warning: Could not extract all circuit properties: {e}")
            num_qubits = 0
            circuit_depth = 0
            gate_count = 0

        # Rigetti's pricing model (example - these are dummy values)
        base_cost = 0.0002  # Base cost per circuit execution
        qubit_cost = 0.00002  # Cost per qubit used
        depth_cost = 0.000002  # Cost per depth unit
        gate_cost = 0.0000005  # Cost per gate

        # Calculate estimated cost
        estimated_cost = (base_cost +
                         (num_qubits * qubit_cost) +
                         (circuit_depth * depth_cost) +
                         (gate_count * gate_cost))

        result = {
            "num_qubits": num_qubits,
            "circuit_depth": circuit_depth,
            "gate_count": gate_count,
            "qubits_used": list(qubits_used) if qubits_used else [],
            "estimated_cost_usd": round(estimated_cost, 8),
            "cost_breakdown": {
                "base_cost": base_cost,
                "qubit_cost_total": round(num_qubits * qubit_cost, 8),
                "depth_cost_total": round(circuit_depth * depth_cost, 8),
                "gate_cost_total": round(gate_count * gate_cost, 8)
            },
            "pyquil_program": str(pyquil_circuit)
        }

        return result

    except Exception as e:
        return {
            "error": f"Failed to analyze Rigetti circuit: {str(e)}",
            "estimated_cost_usd": 0.0,
            "circuit_analyzed": False
        }

def create_sample_pyquil_circuit():
    """Create a sample PyQuil circuit for testing."""
    p = Program()
    p += H(0)
    p += CNOT(0, 1)
    p += RZ(1.5708, 1)  # π/2 rotation
    p += CNOT(1, 2)
    p += MEASURE(0, 0)
    p += MEASURE(1, 1)
    p += MEASURE(2, 2)
    return p

def test_rigetti_analyzer():
    """Test the Rigetti cost analyzer with sample circuits."""

    print("Testing Rigetti Quantum Cost Analyzer...")
    print("=" * 50)

    # Test 1: With a PyQuil circuit directly
    print("\n1. Testing with PyQuil Program:")
    try:
        pyquil_circuit = create_sample_pyquil_circuit()

        # Convert PyQuil to Mitiq for analysis
        mitiq_circuit = convert_to_mitiq(pyquil_circuit)
        result1 = analyze_rigetti_costs(mitiq_circuit)

        print("PyQuil Circuit Analysis:")
        for key, value in result1.items():
            if key != "pyquil_program":  # Skip the long program string
                print(f"  {key}: {value}")
    except Exception as e:
        print(f"  Error with PyQuil test: {e}")

    # Test 2: With a Qiskit circuit
    print("\n2. Testing with Qiskit Circuit:")
    try:
        from qiskit import QuantumCircuit
        qc = QuantumCircuit(3, 3)
        qc.h(0)
        qc.cx(0, 1)
        qc.rz(1.5708, 1)
        qc.cx(1, 2)
        qc.measure_all()

        result2 = analyze_rigetti_costs(qc)

        print("Qiskit Circuit Analysis:")
        for key, value in result2.items():
            if key != "pyquil_program":  # Skip the long program string
                print(f"  {key}: {value}")
    except Exception as e:
        print(f"  Error with Qiskit test: {e}")

    return result1 if 'result1' in locals() else None, result2 if 'result2' in locals() else None

# Alternative version that works directly with PyQuil programs
def analyze_rigetti_costs_direct(pyquil_program):
    """
    Direct analysis of PyQuil Program objects.

    Args:
        pyquil_program: PyQuil Program object

    Returns:
        dict: Cost analysis results
    """
    try:
        # Get circuit characteristics directly from PyQuil
        qubits_used = pyquil_program.get_qubits()
        num_qubits = len(qubits_used) if qubits_used else 0

        # Count instructions
        num_instructions = len(pyquil_program.instructions)

        # Count gates (exclude DECLARE and MEASURE instructions)
        gate_instructions = [instr for instr in pyquil_program.instructions
                           if hasattr(instr, 'name') and
                           instr.name not in ['DECLARE', 'MEASURE']]
        gate_count = len(gate_instructions)

        # Pricing model
        base_cost = 0.0002
        qubit_cost = 0.00002
        instruction_cost = 0.000002
        gate_cost = 0.0000005

        estimated_cost = (base_cost +
                         (num_qubits * qubit_cost) +
                         (num_instructions * instruction_cost) +
                         (gate_count * gate_cost))

        return {
            "num_qubits": num_qubits,
            "total_instructions": num_instructions,
            "gate_count": gate_count,
            "qubits_used": list(qubits_used) if qubits_used else [],
            "estimated_cost_usd": round(estimated_cost, 8),
            "program_length": len(str(pyquil_program))
        }

    except Exception as e:
        return {
            "error": f"Failed to analyze PyQuil program: {str(e)}",
            "estimated_cost_usd": 0.0
        }

# Example usage
if __name__ == "__main__":
    # Test the analyzer
    test_results = test_rigetti_analyzer()

    # Test direct PyQuil analysis
    print("\n3. Testing Direct PyQuil Analysis:")
    sample_program = create_sample_pyquil_circuit()
    direct_result = analyze_rigetti_costs_direct(sample_program)
    print("Direct PyQuil Analysis:")
    for key, value in direct_result.items():
        print(f"  {key}: {value}")

ImportError: cannot import name 'pyquil_executor' from 'mitiq.interface.mitiq_pyquil' (/usr/local/lib/python3.11/dist-packages/mitiq/interface/mitiq_pyquil/__init__.py)

In [ ]:
from pyquil.diagnostics import get_report
print(get_report())
quilc -S
qvm -S

pyQuil version: 4.16.2
qcs-sdk-python version: 0.21.12
Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Python implementation: cpython
Python implementation version: 3.11.13
Python C API version: 1013
Python executable: /usr/bin/python3
venv prefix: /usr
platform: linux
qcs-sdk-rust version: 0.25.11
rustc version: rustc 1.84.1 (e71f9a9a9 2025-01-27)
features: GRPC_WEB, TRACING, TRACING_CONFIG, TRACING_OPENTELEMETRY
qcs:
  address: https://api.qcs.rigetti.com
  network reachable: true
  authentication: failure:  (no status code): error in refresh_qcs_token: Requested an access token for a configuration without credentials.
quilc:
  address: tcp://127.0.0.1:5555
  version: -
  available: false
qvm:
  address: http://127.0.0.1:5000
  version: -
  available: false
libquil:
  available: false
  quilc version: -
  qvm version: -

   



NameError: name 'quilc' is not defined

In [ ]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
result = analyze_rigetti_costs(qc)

AttributeError: 'QuantumCircuit' object has no attribute 'get_qubits'